### 고래 지갑 트랜잭션 수집 및 분석

In [4]:
from web3 import Web3
import pandas as pd
import json
import requests
import psycopg2 as pg
import numpy as np
from collections import OrderedDict

#### API 세팅

In [5]:
etherscan_apikey = 'ETHERSCANKEY'
infura_endpoints = 'https://mainnet.infura.io/v3/INFURAKEY'

In [6]:
w3 = Web3(Web3.HTTPProvider(infura_endpoints))
w3.isConnected()

True

#### 수집할 블록 범위 설정

In [60]:
latest = w3.eth.get_block('latest')['number']
print('latest block number: ', latest)

latest block number:  14477482


In [61]:
block_count = 50
block_range = range(latest - (block_count+1), latest)

In [62]:
block_data = pd.DataFrame(columns=['block_from', 'block_to'])

In [63]:
for i in block_range:
    block_data = block_data.append({'block_from': i,
                                    'block_to': i+1}, ignore_index=True)
block_data = block_data.iloc[:-1]
print(len(block_data))
block_data.iloc[:5]

50


,block_from,block_to
0,14477431,14477432
1,14477432,14477433
2,14477433,14477434
3,14477434,14477435
4,14477435,14477436


#### 트랜잭션 종류 수집 함수

In [71]:
def get_tx_obj(tx, etherscan_apikey):
    try: 
        abi_endpoint = f"https://api.etherscan.io/api?module=contract&action=getabi&address={tx['to']}&apikey={etherscan_apikey}"
        abi = json.loads(requests.get(abi_endpoint).text)
        contract = w3.eth.contract(address=tx["to"], abi=abi["result"])
        func_obj, _ = contract.decode_function_input(tx["input"])
        obj = func_obj
    except ValueError:
        obj = 'undefined'
    return obj

In [7]:
# 고래 지갑 잔고 threshold
whale_threshold = 500

In [8]:
data = pd.DataFrame(columns=['block_number', 'tx_hash', 'tx_obj', 'source', 'source_label', 'target', 'target_label', 'amount', 'gasfees'])
data

,block_number,tx_hash,tx_obj,source,source_label,target,target_label,amount,gasfees


#### 데이터 수집

In [74]:
for block_num in block_range:
    txs_in_block = w3.eth.getBlock(block_num)['transactions']
    print(f'Block {block_num} has {len(txs_in_block)} transactions / last data length: {len(data)}')
    for i in txs_in_block:
        tx_hash = i.hex()
        tx = w3.eth.get_transaction(tx_hash)
        try:
            from_balance = w3.eth.getBalance(tx['from'])/10**18
            to_balance = w3.eth.getBalance(tx['to'])/10**18
        except TypeError:
            print(f'TypeError occurred at tx {tx_hash}')
            
        if from_balance >= whale_threshold or to_balance >= whale_threshold:
            
            amount = 0 if tx['value'] == 0 or tx['value'] == 1 else tx['value']/10**18
            source_label = 'whale' if from_balance >= whale_threshold else 'normal'
            target_label = 'whale' if to_balance >= whale_threshold else 'normal'
            obj = get_tx_obj(tx, etherscan_apikey)
            try:
                data = data.append({'block_number': block_num,
                                    'tx_hash': tx_hash,
                                    'tx_obj': obj,
                                    'source': tx['from'],
                                    'source_label': source_label,
                                    'target': tx['to'],
                                    'target_label': target_label,
                                    'amount': amount,
                                    'gasfees': tx['gasPrice']/(10**18),
                                    }, ignore_index=True)
            except KeyError:
                print(f'KeyError occurred at tx {tx_hash}')
data

Block 14477431 has 274 transactions / last data length: 0
Block 14477432 has 306 transactions / last data length: 31
Block 14477433 has 271 transactions / last data length: 92
Block 14477434 has 297 transactions / last data length: 189
Block 14477435 has 18 transactions / last data length: 284
Block 14477436 has 14 transactions / last data length: 286
Block 14477437 has 263 transactions / last data length: 287
Block 14477438 has 409 transactions / last data length: 327
Block 14477439 has 351 transactions / last data length: 443
Block 14477440 has 49 transactions / last data length: 518
Block 14477441 has 296 transactions / last data length: 530
Block 14477442 has 131 transactions / last data length: 597
Block 14477443 has 468 transactions / last data length: 632
Block 14477444 has 467 transactions / last data length: 796
Block 14477445 has 288 transactions / last data length: 893
Block 14477446 has 31 transactions / last data length: 920
Block 14477447 has 51 transactions / last data l

,block_number,tx_hash,tx_obj,source,source_label,target,target_label,amount,gasfees
0,14477431,0xf8ac3f1d1ba7837ac01fd69959a84e14b5520389ce21...,undefined,0xB10A699f0e9D27273D6D7BdbB7FD44346C7D7076,normal,0x0000000000007F150Bd6f54c40A34d7C3d5e9f56,whale,3.290000e-16,4.723708e-08
1,14477431,0x8d76216a4c178d1bc880bf9ff77d46d4e9b4efa63ed8...,undefined,0xE6d45E57f4740a627c68BB41359567680eFc79df,normal,0x500A746c9a44f68Fe6AA86a92e7B3AF4F322Ae66,whale,1.417869e-01,7.094466e-08
2,14477431,0xe04f7d00aeceb6e70eca3c2d7315af64490dd5ca3098...,undefined,0x224e5B711C192b53C42a27ded78bBC6f5d7d1Bd0,normal,0xCBD6832Ebc203e49E2B771897067fce3c58575ac,whale,1.324540e-01,6.358866e-08
3,14477431,0x58be96231ab76ca60bad804c2069fb8a98c3a41e2e20...,undefined,0xddfAbCdc4D8FfC6d5beaf154f18B778f892A0740,whale,0x829a55cFD0DE6607BcFEaF6eD8bFd69BF2a93394,normal,1.033580e-01,5.623708e-08
4,14477431,0x1b7e6d65256479c95039e3e9d6428d4127182a73bdb2...,undefined,0xddfAbCdc4D8FfC6d5beaf154f18B778f892A0740,whale,0x9E6b85A906596fee9d7eA7E3Ce94Eb07707AEcbE,normal,9.164425e-02,5.623708e-08
...,...,...,...,...,...,...,...,...,...
2239,14477481,0x22de51d9147f2b64e1f10c24c732ddcf548fab4262d5...,undefined,0xeB2629a2734e272Bcc07BDA959863f316F4bD4Cf,whale,0xB9377D4b12883b4D15f200eF6eb606AE5797b4c3,normal,1.482622e-01,5.890841e-08
2240,14477481,0xc71efd04bdc486d985ef25169cf2bbcc6d376c15c6d2...,"<Function transfer(address,uint256)>",0xDFd5293D8e347dFe59E90eFd55b2956a1343963d,whale,0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE,normal,0.000000e+00,5.890841e-08
2241,14477481,0xf5ca7fc59e9eb42b4fa7168bf066f4c65e828968f429...,undefined,0x3cD751E6b0078Be393132286c442345e5DC49699,whale,0x6cfAbDACA0f30c2433A02018ADC2ca37dD827548,normal,1.011365e-02,5.890841e-08
2242,14477481,0x1eef2efb3e2d1584470dea95c4a8e958daa61ff37295...,"<Function mint(uint256,tuple)>",0x5C4ccf80031d3B0bC7256990aC08804A907b3060,normal,0xe182AF6Be923b29f6A53855d5571fDD96B21D93A,whale,6.900000e-01,5.840841e-08


In [75]:
# data.to_csv('whale_tx_220329.csv', index=False)

#### DB 연결

In [26]:
conn = pg.connect(database='dbname',user='username', password='password', host='192.168.xx.xx', port='xxxx')
conn.autocommit = True
curs = conn.cursor()

#### 경로 설정

In [27]:
sql = "CREATE graph eth_test2"
curs.execute(sql)

In [28]:
sql="SET graph_path = eth_test2"
curs.execute(sql)

#### 레이블 생성

In [29]:
node_labels = ['block', 'transaction', 'wallet']

In [30]:
for label in node_labels:
    query = f"CREATE vlabel {label}"
    curs.execute(query)

In [31]:
edge_labels = ['Connected_with', 'Occur', 'Send', 'Receive']

In [32]:
for label in edge_labels:
    query = f"CREATE elabel {label}"
    curs.execute(query)

#### 블록 노드 생성

In [33]:
for i in range(len(block_data)):
    query = "MERGE (n:block {block_number: %d })"%(block_data.iloc[i]['block_from'])
    curs.execute(query)

#### 트랜잭션, 지갑 노드 생성

In [34]:
for i in range(len(data)):
    query1 = "MERGE (a:transaction {{tx_hash: '{}', amount: {} }})".format(data.iloc[i]['tx_hash'], data.iloc[i]['amount'])
    curs.execute(query1)
    query2 = "MERGE (a:wallet {{wallet_address: '{}', wallet_label: '{}' }})".format(data.iloc[i]['source'], data.iloc[i]['source_label'])
    curs.execute(query2)
    query3 = "MERGE (a:wallet {{wallet_address: '{}', wallet_label: '{}' }})".format(data.iloc[i]['target'], data.iloc[i]['target_label'])
    curs.execute(query3)

#### 블록-블록 엣지 생성

In [35]:
for i in range(len(block_data)):
    query = "MATCH (a:block {{block_number: {} }}) \
             MATCH (b:block {{block_number: {} }}) \
             MERGE (a)-[r:Connected_with]->(b)".format(\
             block_data.iloc[i]['block_from'], block_data.iloc[i]['block_to'])
    curs.execute(query)

#### 블록-트랜잭션 엣지 생성

In [36]:
for i in range(len(data)):
    query = "MATCH (a:block {{block_number: {} }}) \
             MATCH (b:transaction {{tx_hash: '{}' }}) \
             MERGE (a)-[r:Occur]->(b)".format(\
             data.iloc[i]['block_number'], data.iloc[i]['tx_hash'])
    curs.execute(query)

#### 트랜잭션-지갑 엣지 생성

In [37]:
for i in range(len(data)):
    query = "MATCH (a:wallet {{wallet_address: '{}' }}) \
             MATCH (b:transaction {{tx_hash: '{}' }}) \
             MERGE (a)-[r:Send]->(b)".format(\
             data.iloc[i]['source'], data.iloc[i]['tx_hash'])
    curs.execute(query)

In [38]:
for i in range(len(data)):
    query =  "MATCH (a:transaction {{tx_hash: '{}' }}) \
              MATCH (b:wallet {{wallet_address: '{}' }}) \
              MERGE (a)-[r:Receive]->(b)".format(\
              data.iloc[i]['tx_hash'], data.iloc[i]['target'])
    curs.execute(query)